<a href="https://colab.research.google.com/github/AmiraliEsi83/The-CIFAR-10-dataset/blob/main/Vgg19OnCifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow.keras as tk
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Dense, Flatten, Dropout
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential

In [ ]:
(x_train, y_train), (x_test, y_test) = tk.datasets.cifar10.load_data()

In [ ]:
x_train.shape

In [ ]:
y_train

In [ ]:
y_train.shape

In [ ]:
classes = np.unique(y_train)

In [ ]:
len(classes)

In [ ]:
x_train[22].shape

In [ ]:
plt.imshow(x_train[22]);

In [ ]:
x_train[22]

In [ ]:
type(x_train)

In [ ]:
x_train = x_train/255

In [ ]:
type(x_train)

In [ ]:
plt.imshow(x_train[22]);

In [ ]:
x_train

In [ ]:
y_train[0].shape

In [ ]:
x_test = x_test/255

In [ ]:
w_grid = 15
l_grid = 15

fig, axes = plt.subplots(w_grid,l_grid, figsize=(17,17))
axes = axes.ravel()
n_training = len(x_train)
for i in np.arange(0, w_grid * l_grid):
  index = np.random.randint(0, n_training)
 
  axes[i].imshow(x_train[index])
  axes[i].set_title(y_train[index], fontsize=10)
  axes[i].axis('off')
fig.tight_layout()

In [ ]:
vgg19_model = VGG19(weights='imagenet',
                    include_top=False, 
                    classes=10,
                    input_shape=(32,32,3)# input: 32x32 images with 3 channels -> (32, 32, 3) tensors.
                   )

In [ ]:
model = Sequential()
for layer in vgg19_model.layers:
    model.add(layer)

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Dropout
model.add(Flatten())
model.add(Dense(4096, activation='relu', name='hidden1'))
# model.add(Dropout(0.4))
# model.add(Dense(4096, activation='relu', name='hidden2'))
# model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax', name='predictions'))

model.summary()

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size=0.2, random_state=12345)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs=100, 
                    validation_data=(x_validate, y_validate))

In [ ]:
evaluation = model.evaluate(x_test,y_test)
print(f"Test accuracy: {evaluation[1]*100:.2f}%")

In [ ]:
predicted_classes = model.predict(x_test)
len(predicted_classes)

In [ ]:
print(predicted_classes[0])
y_test[0]

In [ ]:
np.argmax(predicted_classes[0])

In [ ]:
y_test[0]

In [ ]:
fig, axes = plt.subplots(5,5,figsize=(12,12))
axes = axes.ravel()
print(axes.shape)
x_test[0].shape
for i in np.arange(0,25):
  axes[i].imshow(x_test[i].reshape(28,28))
  axes[i].set_title(f"Prediction Class = {np.argmax(predicted_classes[i]):.1f}\nTrue class = {y_test[i]}")


  axes[i].axis('off')

plt.subplots_adjust(wspace=0.5)

In [ ]:
from vis.utils import utils

In [ ]:
# swap softmax layer with linear layer 
layer_idx = utils.find_layer_idx(model, 'predictions')
model.layers[-1].activation = tf.keras.activations.linear
model = utils.apply_modifications(model)

In [ ]:
!pip install tf-keras-vis

In [ ]:
from matplotlib import cm
from tf_keras_vis.gradcam import Gradcam

In [ ]:
gradCam = Gradcam(model, clone =True)

In [ ]:
from matplotlib.pyplot import plt

In [ ]:
from tf_keras_vis.utils.scores import CategoricalScore

score = CategoricalScore([3, 8, 8 , 0])

In [ ]:
input_classes = ['cat', 'ship', 'ship', 'airplain']

In [ ]:
type(x_test)

In [ ]:
images = [x_test[0], x_test[1], x_test[2], x_test[3]]

In [ ]:
y_test[3]

In [ ]:
plt.imshow(x_test[100])
# y_test[1]

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:
input_images = preprocess_input(images)

In [ ]:
input_images

In [ ]:
# plt.imshow(x_test[1])
y_test[3]

Generate Heatmap

In [ ]:
cam = gradCam(score, input_images, penultimate_layer=-1)

show generated images

In [ ]:
f, ax = plt.subplots(nrows = 1, ncols = 4, figsize=(12,4))
for i, img_class in enumerate(input_classes):
    heatmap = np.uint8(cm.jet(cam[i])[..., :4] * 255)
    ax[i].set_title(img_class, fontsize=16)
    ax[i].imshow(x_test[i])
    ax[i].imshow(heatmap, cmap='jet', alpha=0.2) 
    ax[i].axis('off')
plt.tight_layout()
plt.show()